In [2]:
import json
from http.server import BaseHTTPRequestHandler, HTTPServer
import requests
from datetime import datetime

HOST = "0.0.0.0"
PORT = 8000
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

class ClothingRequest:
    def __init__(self, localizacion, ropa, hora, temperatura):
        self.localizacion = localizacion
        self.ropa = ropa
        self.hora = hora
        self.temperatura = temperatura

class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def _set_headers(self, status=200, content_type="application/json"):
        self.send_response(status)
        self.send_header("Content-type", content_type)
        self.end_headers()

    def do_GET(self):
        if self.path == "/":
            self._set_headers(200, "text/plain; charset=utf-8")
            self.wfile.write("API de recomendación de ropa funcionando".encode("utf-8"))
        elif self.path == "/health":
            self._set_headers()
            response = {"status": "OK", "timestamp": datetime.utcnow().isoformat()}
            self.wfile.write(json.dumps(response, ensure_ascii=False).encode("utf-8"))
        else:
            self._set_headers(404)
            self.wfile.write(json.dumps({"error": "Not Found"}, ensure_ascii=False).encode("utf-8"))

    def do_POST(self):
        if self.path != "/recommend":
            self._set_headers(404)
            self.wfile.write(json.dumps({"error": "Not Found"}, ensure_ascii=False).encode("utf-8"))
            return

        content_length = int(self.headers.get("Content-Length", 0))
        body_bytes = self.rfile.read(content_length)

        try:
            body = body_bytes.decode("utf-8")
        except UnicodeDecodeError:
            body = body_bytes.decode("cp1252")
        
        body = body.strip()
        print(f"[{datetime.utcnow().isoformat()}] POST recibido: {body}")

        try:
            data = json.loads(body)
            localizacion = data.get("localizacion", "")
            ropa = data.get("ropa", "")
            hora = data.get("hora", "")
            temperatura = data.get("temperatura", "")
            
            req = ClothingRequest(localizacion, ropa, hora, temperatura)

            payload = {
                "model": "deepseek-v3.1:671b-cloud",
                "messages": [
                    {
                        "role": "system",
                        "content": (
                            "Eres un asistente experto en clima, hora y vestimenta. "
                            "Recomienda al usuario si su ropa es adecuada. No le digas la ropa que lleva puesta ni el outfit exacto, pero sí si debería cambiar algo."
                        )
                    },
                    {
                        "role": "user",
                        "content": (
                            f"Localización: {req.localizacion}\n"
                            f"Ropa que lleva: {req.ropa}\n"
                            f"Hora: {req.hora}\n"
                            f"Temperatura: {req.temperatura}\n"
                            "Dime si va adecuado, demasiado abrigado o con poca ropa, "
                            "y sugiere cambios si es necesario."
                        )
                    }
                ]
            }

            try:
                res = requests.post(OLLAMA_URL, json=payload, timeout=15)
                res.raise_for_status()
                result = res.json()

                choices = result.get("choices")
                if choices and "message" in choices[0]:
                    content = choices[0]["message"].get("content", "No puedo dar una recomendación ahora.")
                else:
                    content = "No puedo dar una recomendación ahora."

            except requests.RequestException as e:
                print(f"Error conectando a Ollama: {e}")
                content = (
                    f"Simulación: Con {req.ropa} a las {req.hora} en {req.localizacion}, "
                    f"con temperatura de {req.temperatura}, probablemente vas demasiado abrigado o con poca ropa."
                )

            response = {
                "id": f"clothing-recommend-{int(datetime.utcnow().timestamp())}",
                "object": "clothing.recommendation",
                "created": int(datetime.utcnow().timestamp()),
                "model": "deepseek-v3.1:671b-cloud",
                "recommendation": content
            }

            self._set_headers()
            # <-- Cambio aquí
            self.wfile.write(json.dumps(response, ensure_ascii=False).encode("utf-8"))

        except Exception as e:
            print(f"Error interno del servidor: {e}")
            self._set_headers(500)
            self.wfile.write(json.dumps({"error": str(e)}, ensure_ascii=False).encode("utf-8"))

if __name__ == "__main__":
    server = HTTPServer((HOST, PORT), SimpleHTTPRequestHandler)
    print(f"Servidor corriendo en http://{HOST}:{PORT}")
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        print("\nServidor detenido por el usuario")
        server.server_close()


Servidor corriendo en http://0.0.0.0:8000


C:\Users\andsa\AppData\Local\Temp\ipykernel_15048\292026889.py:50: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().isoformat()}] POST recibido: {body}")


[2026-01-09T19:38:52.944008] POST recibido: {"localizacion": "Canary Islands", "ropa": "Chaqueta (24.4%), Vaqueros (28.2%), Cholas/Bailarinas (53.5%), Deportivas (69.0%)", "hora": "19:38", "temperatura": "11.5  \u00b0C"}


C:\Users\andsa\AppData\Local\Temp\ipykernel_15048\292026889.py:104: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "id": f"clothing-recommend-{int(datetime.utcnow().timestamp())}",
C:\Users\andsa\AppData\Local\Temp\ipykernel_15048\292026889.py:106: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created": int(datetime.utcnow().timestamp()),
127.0.0.1 - - [09/Jan/2026 19:38:57] "POST /recommend HTTP/1.1" 200 -


[2026-01-09T19:39:42.801586] POST recibido: {"localizacion": "Canary Islands", "ropa": "Chaqueta (12.3%), Vaqueros (17.4%), Zuecos (58.9%), Zuecos (46.3%)", "hora": "19:39", "temperatura": "11.5  \u00b0C"}


127.0.0.1 - - [09/Jan/2026 19:39:48] "POST /recommend HTTP/1.1" 200 -



Servidor detenido por el usuario
